In [1]:
from uuid import uuid4
import pandas as pd
from langchain_core.documents import Document
import os
from model_config import VietnameseEmbeddings
from elasticsearch import Elasticsearch
# Kết nối tới Elasticsearch
es = Elasticsearch(
    "https://my-elasticsearch-project-fc9fd1.es.ap-southeast-1.aws.elastic.cloud:443",
    api_key="aXY4NkpKUUJaNVNfUEdnZHdVZ186MExmVk1iclZRWWlrS1hpeDRhOWRGUQ=="
)
index = "base"
embeddings = VietnameseEmbeddings()
excel = r"D:\DATN\QA_System\eval\testset\final_dataset_qa.xlsx"
chunks = pd.read_excel(excel)

c:\Users\PC\anaconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Initializing Vietnamese embedding model: keepitreal/vietnamese-sbert


c:\Users\PC\anaconda3\envs\rag\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
import json
import pandas as pd
from elasticsearch import Elasticsearch
from tqdm import tqdm
import time
def search(query, top_k=1, k_vector=50):
    # Mã hóa truy vấn thành vector
    embedding = embeddings.embed_query(query)
    time.sleep(0.05)
    # Thực hiện hybrid search với RRF (Reciprocal Rank Fusion)
    result = es.search(
        index=index,
        retriever = {
                "rrf": {
                    "retrievers": [
                        {
                            "standard": {
                                "query": {
                                    "match": {
                                        "text": query
                                    }
                                }
                            }
                        },
                        {
                            "knn": {
                                "field": "vector",
                                "query_vector": embedding,
                                "k": k_vector,
                                "num_candidates": k_vector * 2
                            }
                        }
                    ],
                    "rank_window_size": k_vector,
                    "rank_constant": 80
                }
            },     size = top_k

    )
    
    # Trích xuất kết quả
    res = [hit['_source']["metadata"]["source"] for hit in result['hits']['hits']]
    
    return res

def evaluate(excel_path, top_k_values=[1, 5, 10, 20]):
    chunk = pd.read_excel(excel_path)
    total_questions = len(chunk)
    
    # Khởi tạo bộ đếm điểm
    hits = {k: 0 for k in top_k_values}
    mrr = {k: 0 for k in top_k_values}
    
    # Add progress bar
    progress_bar = tqdm(chunk.iterrows(), total=total_questions, desc="Evaluating questions")
    
    for _, row in progress_bar:
        question = row['question']
        metadata = row['metadata']
        if isinstance(metadata, str):
            metadata = metadata.replace("'", "\"")
            try:
                metadata = json.loads(metadata)
            except json.JSONDecodeError:
                continue
        correct_source = metadata
        

        # Update progress bar description with current question
        progress_bar.set_description(f"Processing: {question[:30]}...")
        
        # Thực hiện tìm kiếm
        search_results = search(question, top_k=max(top_k_values))

        # Kiểm tra kết quả và tính điểm
        sources = [result for result in search_results]
        # print("correct source:",correct_source)
        # print("search_results:",search_results)
        # print("sources:",sources)
        for k in top_k_values:
            if correct_source in sources[:k]:
                hits[k] += 1
                rank = sources.index(correct_source) + 1
                mrr[k] += 1 / rank if rank <= k else 0
        
        # Update progress bar postfix with current stats
        progress_bar.set_postfix({f'Hit@{max(top_k_values)}': f'{(hits[max(top_k_values)] / (progress_bar.n + 1)):.2f}'})
    
    # Close progress bar
    progress_bar.close()
    
    # Tính tỷ lệ điểm
    hit_ratios = {k: hits[k] / total_questions for k in top_k_values}
    mrr_scores = {k: mrr[k] / total_questions for k in top_k_values}
    
    # Ghi kết quả vào Excel với timestamp
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    output_file = f"evaluation_results_{timestamp}.xlsx"
    result_df = pd.DataFrame({
        'Hit@k': [f"Hit@{k}" for k in top_k_values],
        'Hit Ratio': [hit_ratios[k] for k in top_k_values],
        'MRR': [mrr_scores[k] for k in top_k_values]
    })
    
    result_df.to_excel(output_file, index=False)
    return hit_ratios, mrr_scores

# Chạy đánh giá
excel_path = excel  # Đường dẫn tới file Excel
print("Bắt đầu đánh giá...")
hit_ratios, mrr_scores = evaluate(excel_path)

# In kết quả
print("\nKết quả đánh giá:")
for k in sorted(hit_ratios.keys()):
    print(f"Hit@{k}: {hit_ratios[k]:.4f}")
    print(f"MRR@{k}: {mrr_scores[k]:.4f}")

Bắt đầu đánh giá...


Processing:  Theo số liệu tháng 01/2021, T...: 100%|██████████| 2560/2560 [10:51<00:00,  3.93it/s, Hit@20=0.95]


Kết quả đánh giá:
Hit@1: 0.5016
MRR@1: 0.5016
Hit@5: 0.8137
MRR@5: 0.6278
Hit@10: 0.8934
MRR@10: 0.6385
Hit@20: 0.9535
MRR@20: 0.6427


In [25]:
import os
import pandas as pd

# Thư mục chứa các file Excel
folder_path = r"D:\DATN\chatbot\code"

# Lấy danh sách các file Excel trong thư mục
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

# Danh sách để chứa kết quả từ các file
all_results = []

# Đọc mỗi file Excel và tổng hợp kết quả
for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path)
    
    # Lấy các giá trị Hit@k và MRR từ file Excel
    hit_ratios = df[df['Hit@k'].str.startswith('Hit@')]['Hit Ratio'].values
    mrr_scores = df[df['Hit@k'].str.startswith('Hit@')]['MRR'].values
    
    # Gắn kịch bản (tên file) vào dữ liệu
    result = {
        'Scenario': file,
        'Hit@1': hit_ratios[0] if len(hit_ratios) > 0 else None,
        'MRR@1': mrr_scores[0] if len(mrr_scores) > 0 else None,
        'Hit@5': hit_ratios[1] if len(hit_ratios) > 1 else None,
        'MRR@5': mrr_scores[1] if len(mrr_scores) > 1 else None,
        'Hit@10': hit_ratios[2] if len(hit_ratios) > 2 else None,
        'MRR@10': mrr_scores[2] if len(mrr_scores) > 2 else None,
        'Hit@20': hit_ratios[3] if len(hit_ratios) > 3 else None,
        'MRR@20': mrr_scores[3] if len(mrr_scores) > 3 else None,
    }
    
    all_results.append(result)

# Chuyển đổi danh sách kết quả thành DataFrame
combined_df = pd.DataFrame(all_results)

# Sắp xếp lại cột theo thứ tự để Hit và MRR xen kẽ
combined_df = combined_df[['Scenario', 'Hit@1', 'MRR@1', 'Hit@5', 'MRR@5', 'Hit@10', 'MRR@10', 'Hit@20', 'MRR@20']]

# Lưu kết quả tổng hợp vào một file Excel mới
combined_df.to_excel(r"D:\DATN\chatbot\code\total_results_summary_1_5_10_20.xlsx", index=False)

# In ra kết quả tổng hợp
print("Kết quả tổng hợp đã được lưu vào total_results_summary_1_5_10_20.xlsx.")


Kết quả tổng hợp đã được lưu vào total_results_summary_1_5_10_20.xlsx.


In [6]:
def search(query, top_k=1, k_vector=50):
    # Mã hóa truy vấn thành vector
    embedding = embeddings.embed_query(query)
    
    # Thực hiện hybrid search với RRF (Reciprocal Rank Fusion)
    result = es.search(
        index=index,
        retriever = {
                "rrf": {
                    "retrievers": [
                        {
                            "standard": {
                                "query": {
                                    "match": {
                                        "text": query
                                    }
                                }
                            }
                        },
                        {
                            "knn": {
                                "field": "vector",
                                "query_vector": embedding,
                                "k": k_vector,
                                "num_candidates": k_vector * 2
                            }
                        }
                    ],
                    "rank_window_size": k_vector,
                    "rank_constant": 60
                }
            },     size = top_k

    )
    
    # Trích xuất kết quả
    res = [hit['_source']["metadata"]["source"] for hit in result['hits']['hits']]
    
    return res
search("Đại học bách khoa hà nội")

[{'source': 'Quy định Học bổng KKHT 2023.txt', 'id': '0'}]